In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
import itertools

In [ ]:
def data_capturer(path):
  df = pd.read_excel(path, sheet_name=0, usecols="BE:BW", skiprows=8, nrows=45)
  rowTags = pd.read_excel(path , sheet_name=0 , usecols='A' , skiprows = 8 , nrows = 45)
  df = pd.concat([rowTags , df] , axis = 1)
  dictoflist = {}
  for i , r in df.iterrows():
    dictoflist[r[0]]= r[1:]
  df = pd.DataFrame(dictoflist).reset_index()
  return df
df = data_capturer(r"C:\Users\mrzdu\OneDrive\Masaüstü\ME MYSELF & I\BOGAZICI\JEDI Projects\Image_Electric_Modelling\parameters_stuff\torbali_veriler.xls")

C:\Users\mrzdu\AppData\Local\Temp\ipykernel_23436\2144331640.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  dictoflist[r[0]]= r[1:]


In [ ]:
columns = df.columns[1:]
results = [adfuller(df[c]) for c in columns]

In [ ]:
ADF_statistics = [float(r[0]) for r in results]
p_values = [float(r[1]) for r in results]

In [ ]:
differencing_needed = columns[np.array(p_values) >0.05]
differencing_not_needed = columns[np.array(p_values) <= 0.05]

In [ ]:
df_differenced = {}
for c in differencing_needed:
  df_differenced[c] = df[c].astype(float).diff().dropna()

In [ ]:
df_differenced = pd.DataFrame(df_differenced)
df_not_differenced =df.loc[1:,differencing_not_needed]

In [ ]:
p = range(0,3)
d = range(0,2)
q = range(0,3)
pdq = [itertools.product(p,d,q)]
best_score , best_cfg = float("inf") , None
for parameters in pdq:
  try:
    model = ARIMA(df_differenced['26G1'] , order = parameters)
    model_fit = model.fit()
    if model_fit.aic < best_score:
      best_score ,best_cfg = model_fit.aic , param
  except:
    continue
print(best_cfg , best_score)

In [ ]:
def optimizer(input_data):
  best_score , best_cfg = float("inf") , None
  for p in range(0,3):
    for d in range(0,2):
      for q in range(0,3):
        try:
          model = ARIMA(input_data , order = (p,d,q))
          model_fit = model.fit()
          if model_fit.aic < best_score:
            best_score,best_cfg = model_fit.aic , (p,d,q)
        except:
          continue
  return best_score , best_cfg


In [ ]:
model = ARIMA(df_differenced['26G1'] , order = ( 0 , 0 , 0))

C:\Users\mrzdu\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\mrzdu\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\mrzdu\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


In [ ]:
model_bests_params = [optimizer(df_differenced[e]) for e in differencing_needed]

Görüntülenen çıkış son 5000 satıra kısaltıldı.
C:\Users\mrzdu\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\mrzdu\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\mrzdu\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\mrzdu\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\mrzdu\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and

In [ ]:
model_bests_params_non_differenced = [optimizer(df_not_differenced[e].astype(float)) for e in differencing_not_needed]
model_bests_params_non_differenced

C:\Users\mrzdu\anaconda3\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
C:\Users\mrzdu\anaconda3\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
C:\Users\mrzdu\anaconda3\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
C:\Users\mrzdu\anaconda3\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
C:\Users\mrzdu\anaconda3\Lib\s

[(24.934530767429898, (1, 0, 2)),
 (39.37044344778941, (2, 1, 1)),
 (80.4505969272611, (1, 1, 0))]

In [ ]:
model_bests_params_ordered = model_bests_params + model_bests_params_non_differenced

In [ ]:
differencing_needed
model_cols_ordered = [e for e in differencing_needed] + [e for e in differencing_not_needed]

In [ ]:
#model_bests_params_ordered[i][1] will be order= in the future
#model_cols_ordered[i] dftrain[] inputu olacak.


In [ ]:
dft = pd.concat([df_differenced,df_not_differenced] , axis = 1)

In [ ]:
dft

,26Y1,26Y3,26Y4,26Y5,26Y7,26Y8,26Y9,26Y10,26Y11,26Y12,...,26S8,26S9,26S10,26GSAN1,26GAVM1,26GÖ1,26GÖ2,26Y2,26Y6,26H1
1,-0.717240,-1.899421,-2.862253,-1.142605,-0.302667,-0.618355,-0.781622,-0.913749,0.267719,-0.675608,...,-0.383224,0.183644,1.846936,-1.016363,-2.811726,4.052853,-3.373873,72.703986,60.56319,62.109464
2,0.765330,2.038818,1.254092,1.705255,2.310607,0.717723,0.917699,1.902062,1.866526,1.063701,...,2.491446,-0.556246,1.022624,2.194100,2.101024,6.103788,-0.438508,73.581494,63.615837,60.735357
3,-0.401195,-2.345499,1.612483,-0.836981,-0.693537,-0.788023,-0.537158,-0.273572,0.073153,-0.702080,...,-0.844200,1.182415,3.285423,-0.732403,0.233591,-5.500764,-3.343386,73.314276,63.666844,61.680018
4,1.027128,4.012839,-0.388647,1.020882,1.701214,0.781473,0.649445,2.018472,2.358492,0.938681,...,2.732383,-1.734983,-1.765571,1.075117,1.794450,5.566759,6.545504,74.766741,65.350628,58.683552
5,-1.781096,-3.422427,-1.990793,-1.311338,-1.076213,-0.550463,-0.677311,-0.777357,-1.066158,-0.028642,...,-3.782455,-5.583553,-0.774143,-1.374541,-3.380108,-8.039716,-7.386627,73.262996,64.471325,62.806222
6,1.662350,2.925508,3.012696,2.211975,2.367197,0.561787,0.607443,1.791466,3.470401,0.011220,...,2.719615,7.642858,3.776364,2.702499,3.087109,9.011559,9.310865,74.561336,66.85242,63.578119
7,-1.086990,-2.010419,-3.499854,-1.286450,-1.215999,-0.681277,-0.678619,-0.915988,-1.562530,-0.721697,...,-4.390666,-0.902062,-1.935342,-2.240020,-2.185969,-1.774697,-2.654742,73.396564,65.858097,61.826358
8,0.961885,2.525478,4.584481,0.980781,1.069653,0.565693,0.477893,0.709955,2.300941,0.978892,...,2.856395,-0.336108,2.105116,2.834298,1.564040,2.418473,1.455051,74.052579,67.253322,63.795984
9,-0.894617,-2.250086,-3.312756,-1.954582,-1.608752,-0.773023,-0.414794,-1.440644,-2.471286,-1.516214,...,-1.695207,-0.776837,0.384614,-2.023802,-2.220993,-3.769452,1.684353,73.141012,66.225891,62.777056
10,0.951209,2.616750,4.170047,2.089825,2.220437,0.786891,0.532605,1.957681,2.337142,1.146486,...,3.152338,1.209093,0.059300,2.805370,1.983681,3.331606,1.495599,74.313793,68.200509,61.635564


In [ ]:
dftrain = dft.iloc[:14,:]
dftest = df.iloc[14:,:]

In [ ]:
model = ARIMA(dftrain[model_cols_ordered[0]].astype(float) , order = model_bests_params_ordered[0][1])
model_fit = model.fit()
predictions = model_fit.forecast(steps = len(dftest))

C:\Users\mrzdu\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\mrzdu\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\mrzdu\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\mrzdu\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\mrzdu\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next versio

'26H1'

In [ ]:
predictions

14   -1.033749
15    0.971004
16   -0.818258
17    0.778676
18   -0.646604
Name: predicted_mean, dtype: float64

In [ ]:
mape = (np.abs(dftest[model_cols_ordered[0]] - predictions ) / dftest[model_cols_ordered[0]]).mean()*100
mae = (np.abs(dftest[model_cols_ordered[0]] - predictions ) / dftest[model_cols_ordered[0]]).mean()*100

KeyError: '26Y1'

In [ ]:
model_bests_params_ordered

[(10.555015721634284, (1, 0, 1)),
 (43.574914648596874, (1, 0, 0)),
 (64.55835109580995, (1, 0, 1)),
 (31.774677811220403, (1, 0, 1)),
 (41.16904937100081, (1, 0, 1)),
 (-1.8196186695275554, (1, 0, 1)),
 (-4.445403998313569, (1, 0, 0)),
 (34.526397078655094, (1, 0, 1)),
 (51.9982522425373, (1, 1, 0)),
 (27.54805796706554, (1, 0, 2)),
 (36.93991809681835, (1, 0, 2)),
 (38.46537351810433, (1, 0, 1)),
 (24.687818986896573, (1, 0, 2)),
 (40.04377436756059, (1, 0, 2)),
 (59.116356442373096, (0, 0, 1)),
 (90.2871392603572, (1, 0, 2)),
 (68.30207685693955, (0, 0, 2)),
 (42.97047526933836, (1, 0, 0)),
 (33.95193823150072, (1, 0, 2)),
 (76.21685556679601, (1, 0, 1)),
 (70.6394090801962, (1, 0, 1)),
 (57.744112305314445, (1, 1, 1)),
 (59.495925458222956, (0, 0, 1)),
 (80.26676385460613, (0, 0, 1)),
 (64.43284323141995, (1, 0, 2)),
 (89.68554431523978, (0, 0, 1)),
 (60.85695501807618, (1, 0, 1)),
 (53.87314738557999, (0, 1, 2)),
 (47.783436018229565, (0, 0, 1)),
 (57.5265768782344, (1, 0, 2)),
 (

In [ ]:
dftrain

,26Y1,26Y3,26Y4,26Y5,26Y7,26Y8,26Y9,26Y10,26Y11,26Y12,...,26S8,26S9,26S10,26GSAN1,26GAVM1,26GÖ1,26GÖ2,26Y2,26Y6,26H1
1,-0.717240,-1.899421,-2.862253,-1.142605,-0.302667,-0.618355,-0.781622,-0.913749,0.267719,-0.675608,...,-0.383224,0.183644,1.846936,-1.016363,-2.811726,4.052853,-3.373873,72.703986,60.56319,62.109464
2,0.765330,2.038818,1.254092,1.705255,2.310607,0.717723,0.917699,1.902062,1.866526,1.063701,...,2.491446,-0.556246,1.022624,2.194100,2.101024,6.103788,-0.438508,73.581494,63.615837,60.735357
3,-0.401195,-2.345499,1.612483,-0.836981,-0.693537,-0.788023,-0.537158,-0.273572,0.073153,-0.702080,...,-0.844200,1.182415,3.285423,-0.732403,0.233591,-5.500764,-3.343386,73.314276,63.666844,61.680018
4,1.027128,4.012839,-0.388647,1.020882,1.701214,0.781473,0.649445,2.018472,2.358492,0.938681,...,2.732383,-1.734983,-1.765571,1.075117,1.794450,5.566759,6.545504,74.766741,65.350628,58.683552
5,-1.781096,-3.422427,-1.990793,-1.311338,-1.076213,-0.550463,-0.677311,-0.777357,-1.066158,-0.028642,...,-3.782455,-5.583553,-0.774143,-1.374541,-3.380108,-8.039716,-7.386627,73.262996,64.471325,62.806222
6,1.662350,2.925508,3.012696,2.211975,2.367197,0.561787,0.607443,1.791466,3.470401,0.011220,...,2.719615,7.642858,3.776364,2.702499,3.087109,9.011559,9.310865,74.561336,66.85242,63.578119
7,-1.086990,-2.010419,-3.499854,-1.286450,-1.215999,-0.681277,-0.678619,-0.915988,-1.562530,-0.721697,...,-4.390666,-0.902062,-1.935342,-2.240020,-2.185969,-1.774697,-2.654742,73.396564,65.858097,61.826358
8,0.961885,2.525478,4.584481,0.980781,1.069653,0.565693,0.477893,0.709955,2.300941,0.978892,...,2.856395,-0.336108,2.105116,2.834298,1.564040,2.418473,1.455051,74.052579,67.253322,63.795984
9,-0.894617,-2.250086,-3.312756,-1.954582,-1.608752,-0.773023,-0.414794,-1.440644,-2.471286,-1.516214,...,-1.695207,-0.776837,0.384614,-2.023802,-2.220993,-3.769452,1.684353,73.141012,66.225891,62.777056
10,0.951209,2.616750,4.170047,2.089825,2.220437,0.786891,0.532605,1.957681,2.337142,1.146486,...,3.152338,1.209093,0.059300,2.805370,1.983681,3.331606,1.495599,74.313793,68.200509,61.635564


In [ ]:
dftrain.iloc[-1]['26Y1']

1.2990850099999989

In [ ]:
dftest

,26Y1,26Y3,26Y4,26Y5,26Y7,26Y8,26Y9,26Y10,26Y11,26Y12,...,26S8,26S9,26S10,26GSAN1,26GAVM1,26GÖ1,26GÖ2,26Y2,26Y6,26H1
15,-1.002703,-1.064778,-1.488560,-0.897044,-0.241784,-0.638788,-0.880415,-0.633764,0.841982,-0.750067,...,-0.462353,6.306098,2.750814,2.413183,-0.376069,-3.547036,-2.003050,73.567831,68.902541,60.26065
16,1.077125,1.237439,0.919081,0.987879,1.498721,0.739084,0.783326,1.502213,3.343194,1.174382,...,1.990690,-1.050489,0.685280,1.302373,0.613654,6.757474,0.647194,74.340729,70.233918,67.501158
17,-0.794049,-1.486356,-1.389751,-1.585576,-0.857142,-0.483180,-0.526091,-0.572153,0.630517,-0.354432,...,-1.003289,1.121860,-2.645538,-0.995131,-1.333549,-11.132838,-3.552481,73.689301,69.142798,63.167968
18,0.811083,0.846079,1.407018,1.383140,0.791727,0.421110,0.506328,1.642907,2.496396,0.274487,...,2.146468,1.947005,1.347894,0.914525,1.049254,3.082170,2.359763,74.012653,70.106613,65.660063


In [ ]:
dftest

,26Y1,26Y3,26Y4,26Y5,26Y7,26Y8,26Y9,26Y10,26Y11,26Y12,...,26S8,26S9,26S10,26GSAN1,26GAVM1,26GÖ1,26GÖ2,26Y2,26Y6,26H1
15,-1.002703,-1.064778,-1.488560,-0.897044,-0.241784,-0.638788,-0.880415,-0.633764,0.841982,-0.750067,...,-0.462353,6.306098,2.750814,2.413183,-0.376069,-3.547036,-2.003050,73.567831,68.902541,60.26065
16,1.077125,1.237439,0.919081,0.987879,1.498721,0.739084,0.783326,1.502213,3.343194,1.174382,...,1.990690,-1.050489,0.685280,1.302373,0.613654,6.757474,0.647194,74.340729,70.233918,67.501158
17,-0.794049,-1.486356,-1.389751,-1.585576,-0.857142,-0.483180,-0.526091,-0.572153,0.630517,-0.354432,...,-1.003289,1.121860,-2.645538,-0.995131,-1.333549,-11.132838,-3.552481,73.689301,69.142798,63.167968
18,0.811083,0.846079,1.407018,1.383140,0.791727,0.421110,0.506328,1.642907,2.496396,0.274487,...,2.146468,1.947005,1.347894,0.914525,1.049254,3.082170,2.359763,74.012653,70.106613,65.660063


In [ ]:
df

,index,26Y1,26Y2,26Y3,26Y4,26Y5,26Y6,26Y7,26Y8,26Y9,...,26S6,26S7,26S8,26S9,26S10,26GSAN1,26GAVM1,26GÖ1,26GÖ2,26H1
0,2015-2.1,73.535567,73.546849,70.036536,69.363545,70.662245,61.184059,64.258036,74.887096,71.771324,...,36.723155,44.546707,46.137859,49.88654,38.963128,14.558549,30.011684,33.00316,59.703933,61.452583
1,2016-1.1,72.818327,72.703986,68.137114,66.501292,69.519641,60.56319,63.955369,74.268741,70.989702,...,36.047894,44.016986,45.754636,50.070184,40.810063,13.542185,27.199959,37.056013,56.33006,62.109464
2,2016-2.1,73.583657,73.581494,70.175932,67.755384,71.224896,63.615837,66.265976,74.986464,71.907401,...,35.269129,46.644088,48.246082,49.513938,41.832687,15.736285,29.300983,43.159802,55.891552,60.735357
3,2017-1.1,73.182462,73.314276,67.830433,69.367867,70.387915,63.666844,65.572439,74.198441,71.370243,...,38.910264,49.105161,47.401881,50.696353,45.118111,15.003882,29.534574,37.659038,52.548166,61.680018
4,2017-2.1,74.20959,74.766741,71.843272,68.97922,71.408797,65.350628,67.273653,74.979914,72.019688,...,36.325262,48.694689,50.134264,48.961371,43.35254,16.078999,31.329024,43.225797,59.09367,58.683552
5,2018-1.1,72.428494,73.262996,68.420845,66.988426,70.097459,64.471325,66.19744,74.429451,71.342377,...,36.185572,46.547335,46.351809,43.377817,42.578396,14.704458,27.948915,35.186081,51.707042,62.806222
6,2018-2.1,74.090844,74.561336,71.346353,70.001122,72.309433,66.85242,68.564636,74.991238,71.94982,...,38.590972,49.405869,49.071424,51.020676,46.35476,17.406957,31.036025,44.19764,61.017907,63.578119
7,2019-1.1,73.003854,73.396564,69.335934,66.501268,71.022983,65.858097,67.348637,74.309961,71.271201,...,38.718114,47.687811,44.680759,50.118613,44.419419,15.166937,28.850055,42.422942,58.363165,61.826358
8,2019-2.1,73.96574,74.052579,71.861412,71.085749,72.003764,67.253322,68.41829,74.875654,71.749093,...,40.22296,50.005415,47.537153,49.782506,46.524535,18.001235,30.414096,44.841415,59.818216,63.795984
9,2020-1.1,73.071123,73.141012,69.611326,67.772993,70.049182,66.225891,66.809539,74.102632,71.334299,...,39.741217,49.071026,45.841946,49.005669,46.90915,15.977432,28.193103,41.071963,61.502569,62.777056


In [ ]:
dftest

,index,26Y1,26Y2,26Y3,26Y4,26Y5,26Y6,26Y7,26Y8,26Y9,...,26S6,26S7,26S8,26S9,26S10,26GSAN1,26GAVM1,26GÖ1,26GÖ2,26H1
14,2022-2.1,74.197048,74.520171,73.539072,70.850119,72.05322,69.917105,70.751318,74.971794,72.157986,...,40.289144,51.849413,48.05485,52.421898,48.546028,16.83549,28.484077,46.623084,64.299691,64.064982
15,2023-1.1,73.194344,73.567831,72.474293,69.361559,71.156177,68.902541,70.509534,74.333006,71.277571,...,43.465628,52.654161,47.592497,58.727996,51.296842,19.248673,28.108008,43.076049,62.296641,60.26065
16,2023-2.1,74.271469,74.340729,73.711733,70.28064,72.144055,70.233918,72.008255,75.07209,72.060896,...,42.668242,53.985092,49.583186,57.677507,51.982122,20.551046,28.721662,49.833522,62.943835,67.501158
17,2024-1.1,73.47742,73.689301,72.225376,68.890889,70.55848,69.142798,71.151114,74.58891,71.534805,...,43.063574,53.315513,48.579897,58.799366,49.336584,19.555914,27.388113,38.700685,59.391354,63.167968
18,2024-2.1,74.288503,74.012653,73.071455,70.297907,71.941619,70.106613,71.942841,75.01002,72.041133,...,44.315696,54.942338,50.726365,60.746371,50.684479,20.470439,28.437367,41.782855,61.751118,65.660063


In [ ]:
dftrain.iloc[-1]['26Y1']

1.2990850099999989

In [ ]:
mape_list = []
mae_list = []
mse_list = []
for i , col in enumerate(model_cols_ordered[:-3]):
  model = ARIMA(dftrain[col].astype(float) , order = model_bests_params_ordered[i][1] )
  model_fit = model.fit()
  predictions = model_fit.forecast(steps = len(dftest))
  mape = (np.abs(dftest[col]  - df.iloc[14][col] - predictions) / dftest[col]).mean()*100
  mae = (np.abs(dftest[col]  - df.iloc[14][col] - predictions)).mean()
  mse = ((dftest[col]  - df.iloc[14][col] - predictions)**2).mean()
  mae_list.append(mae)
  mse_list.append(mse)
  mape_list.append(mape)
for i , col in enumerate(model_cols_ordered[-3:]):
  model = ARIMA(dft[col].astype(float) , order = model_bests_params_ordered[i][1] )
  model_fit = model.fit()
  predictions = model_fit.forecast(steps = len(dftest))
  mape = (np.abs(dftest[col] - predictions) / dftest[col]).mean()*100
  mae = (np.abs(dftest[col] - predictions)).mean()
  mse = (np.abs(dftest[col] - predictions)**2).mean()
  mape_list.append(mape)
  mae_list.append(mae)
  mse_list.append(mse)

C:\Users\mrzdu\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\mrzdu\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\mrzdu\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\mrzdu\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\mrzdu\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next versio

In [ ]:
year_start = 2025
listed_additional_i = []
for i in range(20):
  listed_additional_i.append(str(year_start + i//2) +'-' + str(i%2 + 1) + '.1' )
listed_additional_index = {'index' : np.array(listed_additional_i).astype(np.object_)}

for i , col in enumerate(model_cols_ordered[:-3]):
  model = ARIMA(dft[col].astype(float) , order = model_bests_params_ordered[i][1] )
  model_fit = model.fit()
  predictions = model_fit.forecast(steps = 20)
  listed_additional_index[col + '_forecasted'] = predictions + df.iloc[-1][col]

for i , col in enumerate(model_cols_ordered[-3:]):
  model = ARIMA(dft[col].astype(float) , order = model_bests_params_ordered[i][1] )
  model_fit = model.fit()
  predictions = model_fit.forecast(steps = 20)
  listed_additional_index[col + '_forecasted'] = predictions

forecasts_df = pd.DataFrame(listed_additional_index)

C:\Users\mrzdu\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\mrzdu\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\mrzdu\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\mrzdu\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\mrzdu\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next versio

In [ ]:
forecasts = pd.concat([pd.DataFrame({'index':listed_additional_i}),forecasts_df.reset_index().iloc[:,1:]],axis=1)

In [ ]:
forecasts_df

,index,26Y1_forecasted,26Y3_forecasted,26Y4_forecasted,26Y5_forecasted,26Y7_forecasted,26Y8_forecasted,26Y9_forecasted,26Y10_forecasted,26Y11_forecasted,...,26S8_forecasted,26S9_forecasted,26S10_forecasted,26GSAN1_forecasted,26GAVM1_forecasted,26GÖ1_forecasted,26GÖ2_forecasted,26Y2_forecasted,26Y6_forecasted,26H1_forecasted
18,2025-1.1,73.440163,72.676489,69.782685,70.620884,71.482869,74.374526,71.597255,59.500142,61.828765,...,49.881529,62.775708,52.110647,19.607393,27.445113,31.733062,60.228810,73.488270,69.761373,60.791532
19,2025-2.1,75.110888,73.850363,70.880085,73.404860,73.269910,75.658513,72.518276,61.785227,63.471531,...,52.722350,62.649840,51.278102,21.673403,29.214882,43.864499,63.150778,74.216007,69.445951,63.954525
20,2026-1.1,73.612043,72.740024,70.073139,70.620885,71.496961,74.374527,71.625543,59.500148,61.930108,...,50.024441,62.649840,51.278102,19.914936,27.680682,32.657358,60.951463,73.490855,69.157770,61.090543
21,2026-2.1,74.956691,73.790267,70.666507,73.404858,73.255930,75.658511,72.490857,61.785220,63.376440,...,52.586627,62.649840,51.278102,21.625436,29.010669,43.010624,62.606849,74.213432,68.894479,63.683780
22,2027-1.1,73.750377,72.796868,70.230189,70.620887,71.510831,74.374529,71.652120,59.500155,62.019333,...,50.153336,62.649840,51.278102,19.961594,27.857713,33.446175,61.360868,73.493422,68.653927,61.335693
23,2027-2.1,74.832588,73.736500,70.551024,73.404857,73.242169,75.658510,72.465096,61.785214,63.292719,...,52.464217,62.649840,51.278102,21.580051,28.857203,42.281907,62.298697,74.210875,68.434151,63.461806
24,2028-1.1,73.861712,72.847724,70.315106,70.620888,71.524483,74.374531,71.677090,59.500161,62.097889,...,50.269589,62.649840,51.278102,20.005742,27.990752,34.119371,61.592810,73.495970,68.233357,61.536683
25,2028-2.1,74.732707,73.688396,70.488583,73.404855,73.228624,75.658508,72.440893,61.785207,63.219010,...,52.353813,62.649840,51.278102,21.537108,28.741872,41.660002,62.124119,74.208335,68.049905,63.279816
26,2029-1.1,73.951318,72.893225,70.361021,70.620890,71.537920,74.374533,71.700549,59.500168,62.167051,...,50.374439,62.649840,51.278102,20.047513,28.090731,34.693893,61.724211,73.498500,67.882296,61.701469
27,2029-2.1,74.652320,73.645358,70.454820,73.404854,73.215293,75.658506,72.418155,61.785201,63.154114,...,52.254237,62.649840,51.278102,21.496476,28.655201,41.129253,62.025215,74.205814,67.729164,63.130608


In [ ]:
for colname in model_cols_ordered:
  e = colname
  f = '_forecasted'
  plt.plot( df['index'] , df[e] , c = 'blue' , marker = 'o' ,  markersize = 1 , linewidth = 0.5 , linestyle = ':' , label = 'observation_set')
  plt.ylim(bottom = 0, top = df[e].max() + 15)
  plt.xticks(rotation = 90)
  plt.grid(axis = 'y',linestyle = '--')
  plt.plot(forecasts['index'] , forecasts[e + f] , c = 'red' , marker = 'o' , markersize = 1 , linewidth = 0.5 , linestyle = ':' , label = 'forecast_set')
  plt.legend(loc = 'lower right')
  plt.savefig(r"C:\Users\mrzdu\OneDrive\Masaüstü\ME MYSELF & I\BOGAZICI\JEDI Projects\Image_Electric_Modelling\ARIMA" + f'\\{e}_10_year_fig')
  plt.close(fig = 'all')

In [ ]:

pd.concat([listed_additional_df , forecast_df],axis=0,ignore_index=True)

,index,26Y1_forecasted,26Y3_forecasted,26Y4_forecasted,26Y5_forecasted,26Y7_forecasted,26Y8_forecasted,26Y9_forecasted,26Y10_forecasted,26Y11_forecasted,...,26S8_forecasted,26S9_forecasted,26S10_forecasted,26GSAN1_forecasted,26GAVM1_forecasted,26GÖ1_forecasted,26GÖ2_forecasted,26Y2_forecasted,26Y6_forecasted,26H1_forecasted
0,2025-1.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2025-2.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2026-1.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2026-2.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2027-1.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2027-2.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2028-1.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2028-2.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2029-1.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2029-2.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
p=[]
d=[]
q=[]
aic=[]
for a,param in model_bests_params_ordered:
  p.append(param[0])
  d.append(param[1])
  q.append(param[2])
  aic.append(float(a))


In [ ]:
report_df = pd.DataFrame({'zone':model_cols_ordered,'p':p,'d':d,'q':q,'aic':aic,'test_mape':mape_list,'test_mae':mae_list,'test_mse':mse_list})

report_df.to_excel(r"C:\Users\mrzdu\OneDrive\Masaüstü\ME MYSELF & I\BOGAZICI\JEDI Projects\Image_Electric_Modelling\ARIMA\report_df.xlsx")